In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
import sounddevice as sd
from matplotlib import cm
from PIL import Image
import cv2
import time

# Configuración
DURATION = 0.1  # segundos por bloque
FS = 44100
IMG_HEIGHT = 480
IMG_WIDTH = 1280
SIGNAL_HEIGHT = 80  # altura de la franja de señal
NFFT = 1024
NOVERLAP = int(0.9 * NFFT)  # 90% de solapamiento

# Inicializar imagen del espectrograma
scroll_img = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)

# Crear ventana en pantalla completa
window_name = "Espectrograma y Señal"
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Obtener tamaño de pantalla
screen_res = (cv2.getWindowImageRect(window_name)[2], cv2.getWindowImageRect(window_name)[3])

# Número de ciclos
n_cycles = 500
for _ in range(n_cycles):
    # Captura de audio
    audio = sd.rec(int(DURATION * FS), samplerate=FS, channels=1, dtype='float32')
    sd.wait()
    signal = audio[:, 0]

    # Calcular espectrograma
    f, t, Sxx = spectrogram(signal, fs=FS, nperseg=NFFT, noverlap=NOVERLAP)
    Sxx_log = 10 * np.log10(Sxx + 1e-10)
    Sxx_norm = (Sxx_log - Sxx_log.min()) / (Sxx_log.max() - Sxx_log.min() + 1e-10)

    # Convertir espectrograma en imagen de color
    colormap = cm.inferno(Sxx_norm)
    colormap_img = (colormap[:, :, :3] * 255).astype(np.uint8)
    new_column = np.array(Image.fromarray(colormap_img).resize((10, IMG_HEIGHT - SIGNAL_HEIGHT)))

    # Desplazar imagen hacia la izquierda
    scroll_img[SIGNAL_HEIGHT:, :-10] = scroll_img[SIGNAL_HEIGHT:, 10:]
    scroll_img[SIGNAL_HEIGHT:, -10:] = new_column

    # Dibujar la forma de onda (parte superior)
    wave_section = np.zeros((SIGNAL_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
    normalized_signal = ((signal - signal.min()) / (signal.max() - signal.min() + 1e-10))
    waveform = (1 - normalized_signal) * (SIGNAL_HEIGHT - 1)
    waveform = waveform.astype(np.int32)

    for i in range(len(waveform) - 1):
        x1 = int(i * IMG_WIDTH / len(waveform))
        x2 = int((i + 1) * IMG_WIDTH / len(waveform))
        y1 = waveform[i]
        y2 = waveform[i + 1]
        cv2.line(wave_section, (x1, y1), (x2, y2), (255, 255, 255), 1)

    combined_img = np.vstack((wave_section, scroll_img[SIGNAL_HEIGHT:]))

    # Redimensionar a pantalla completa
    full_screen_img = cv2.resize(combined_img, screen_res)

    # Mostrar en pantalla
    cv2.imshow(window_name, full_screen_img)

    time.sleep(0.01)

    # Salir si se presiona ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()



